In [30]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mtl

%matplotlib inline

In [31]:
def get_data():
    # Read sqlite query results into a pandas DataFrame
    con = sqlite3.connect("/Users/moran/PycharmProjects/keylogger/db/logger.db")
    df = pd.read_sql_query("SELECT * from moves", con)

    # Verify that result of SQL query is stored in the dataframe
    con.close()
    return df

df = get_data()
df

,x,y,timestamp,session_id,event_type
0,150,1482,488.4,86016981349520,move
1,152,1475,496.5,86016981349520,move
2,154,1450,529.0,86016981349520,move
3,155,1448,537.2,86016981349520,move
4,155,1424,778.0,86016981349520,click
...,...,...,...,...,...
2218,29,1724,3206.6,132336280261440,move
2219,29,1724,3206.6,132336280261440,move
2220,29,1724,3206.6,132336280261440,move
2221,29,1724,3206.6,132336280261440,move


In [32]:
sessions = df.value_counts('session_id')
sessions

session_id
51279743651108     668
163766001040587    661
119101341014136    295
61205527402026     226
109177423051620    170
132336280261440    161
86016981349520      42
dtype: int64

In [33]:
df.groupby('session_id').count()

,x,y,timestamp,event_type
session_id,,,,
109177423051620,170,170,170,170
119101341014136,295,295,295,295
132336280261440,161,161,161,161
163766001040587,661,661,661,661
51279743651108,668,668,668,668
61205527402026,226,226,226,226
86016981349520,42,42,42,42


In [34]:
def handle_session(session_id, total_df):
    df = total_df[total_df['session_id'] == session_id]
    df.sort_values('timestamp', inplace=True)
    moves = df[df['event_type'] == 'move']
    moves.reset_index(drop=True, inplace=True)
    clicks = df[df['event_type'] == 'click']
    clicks.reset_index(drop=True, inplace=True)
    # calc dist
    dx = moves["x"].iloc[1:].values - moves["x"].iloc[0:-1].values
    dy = moves["y"].iloc[1:].values - moves["y"].iloc[0:-1].values
    moves['dist'] = np.append([0], np.sqrt(np.power(dx, 2) + np.power(dy, 2)))
    # calc time change 
    dt = moves["timestamp"].iloc[1:].values - moves["timestamp"].iloc[0:-1].values
    moves['dt'] = np.append([0], dt)
    # calc speed 

    moves['speed'] = moves['dist'] / moves['dt']
    moves['speed'].fillna(0, inplace=True)
    
    moves['speed_cat'] = pd.cut(moves['speed'], bins=np.arange(-1,20), labels=np.arange(1,21))

    return moves, clicks

In [35]:
moves, clicks = handle_session("163766001040587", df)

/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/var/folders/cb/j7mnj9p54fgfc1kpt2kpqph40000gn/T/ipykernel_10330/3533241900.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moves['dist'] = np.append([0], np.sqrt(np.power(dx, 2) + np.power(dy, 2)))
/var/folders/cb/j7mnj9p54fgfc1kpt2kpqph40000gn/T/ipykernel_10330/3533241900.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [36]:
def extract_images(moves, clicks):
    plt.rcParams['axes.facecolor'] = 'black'    
    fig, ax = plt.subplots( nrows=1, ncols=1 )   
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)    

    prev_val = None
    rows = []
    cmap = mtl.cm.get_cmap('Reds_r')
    norm = mtl.colors.Normalize(vmin=1, vmax=20)

    for index, row in moves.iterrows():
        if prev_val == row['speed_cat'] or prev_val == None:
            rows.append(index)
        else:
            rows.append(index)
            plt.plot(moves.loc[rows]['x'], moves.loc[rows]['y'], c=cmap(norm(prev_val)))
            rows = [index]
        prev_val = row['speed_cat']

    plt.scatter(clicks['x'], clicks['y'], s=200)
    
    fig.savefig('with_clicks.png', bbox_inches='tight', dpi=300)
    plt.close(fig) 
    
extract_images(moves, clicks)

In [4]:
 import cv2

In [3]:
!pip install opencv-python

     |████████████████████████████████| 46.3 MB 4.5 MB/s eta 0:00:01


In [5]:
image= cv2.imread('images/84376088071389.png')

In [6]:
flippedimage= cv2.flip(image, 0)

In [8]:
cv2.imshow('Vertically Flipped Image', flippedimage)

In [12]:
cv2.imwrite("images/a.png", flippedimage)

True